In [ ]:
#Install Dependencies
!git clone https://github.com/GraySwanAI/nanoGCG.git
%cd nanoGCG
!pip install nanogcg
!pip install datasets

In [ ]:
#Log on to HF
from huggingface_hub import login

# Replace YOUR_HF_TOKEN with your actual Hugging Face token
login("")


In [ ]:
#Load worst prompts from AdvBench Data Set, and then pick a random prompt
from datasets import load_dataset

# Load the full AdvBench dataset
dataset = load_dataset("walledai/AdvBench", split="train")
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import nanogcg
from nanogcg import GCGConfig

# Load one random sample from AdvBench
dataset = load_dataset("walledai/AdvBench", split="train")
keywords = ["kill", "attack", "bomb", "hack", "steal", "murder", "rob"]
def keyword_score(text):
    return sum(kw in text.lower() for kw in keywords)

worst_10 = sorted(dataset, key=lambda x: keyword_score(x["target"]), reverse=True)[:10]

example = random.choice(worst_10)
prompt = example["prompt"]
target = example["target"]

print("🧨 Prompt:", prompt)
print("🎯 Target:", target)

In [ ]:
#Loading the actual model + tokenizer
print("🧨 Prompt:", prompt)
print("🎯 Target:", target)
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

Testing the prompt - Should be denied till here (Some Adv Prompts work because model doesn't block them, shuffle the adv prompt from the above code to find one that doesn't work and attack using nanoGCG)

In [ ]:
adv_prompt = prompt
inputs = tokenizer(adv_prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=200)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n📤 Final Model Output:\n", decoded)

In [ ]:
#Running nanoGCG with only 150 steps
config = GCGConfig(
    num_steps=150,
    search_width=512,
    topk=256,
    n_replace=2,
    buffer_size=10,
    use_mellowmax=True,
    mellowmax_alpha=1.0,
    early_stop=True,
    seed=42,
    filter_ids=False  # disable filtering for max flexibility
)


result = nanogcg.run(
    model,
    tokenizer,
    prompt,
    target,
    config
)

Prompting the surrogate model with Adv Suffix (should work here now)

In [ ]:
print("\n✅ Optimized Suffix:\n", result.best_string)

adv_prompt = prompt + " " + result.best_string
inputs = tokenizer(adv_prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=200)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n📤 Final Model Output:\n", decoded)